In [1]:
import numpy as np
from torch import nn
import torch
from torch.nn.functional import conv_transpose2d
import pytest
from conv2D import conv2d


In [3]:
# %%file conv_transpose.py

import numpy as np
from torch import nn
import torch
from torch.nn.functional import conv_transpose2d
import pytest
from conv2D import conv2d

def myConvT(input_data, weight, bias=None, stride=1, padding=0, output_padding=0, groups=1, dilation=1):
    output = conv2d(input_data, weight, padding=0, dilation=1, stride=1, groups = 1)

    return output


In [4]:
input_data = torch.randn(1, 1, 3, 3)
weight = torch.randn(1, 1, 3, 3)

your_result = myConvT(input_data.numpy(), weight.numpy())
torch_result = torch.nn.functional.conv_transpose2d(input_data, weight).numpy()

print(your_result)
print(torch_result)

[[[[-0.04509263  0.87619458 -0.42913079 -0.50778693  0.87607318]
   [-0.26734793  1.31673771 -0.85584934  2.39083022  3.1351487 ]
   [-0.54947088  0.98111774 -2.54545097  3.74698555  4.08418187]
   [ 0.05541194  0.36245742 -3.83368918  1.118823    1.76252534]
   [-0.14232606  1.15755347 -1.6258672   0.47553246 -0.09795115]]]]
[[[[-0.04509263  0.8761946  -0.4291308  -0.5077869   0.8760732 ]
   [-0.26734793  1.3167377  -0.8558493   2.39083     3.1351488 ]
   [-0.5494709   0.9811177  -2.5454512   3.7469852   4.084182  ]
   [ 0.05541194  0.36245742 -3.8336892   1.1188229   1.7625253 ]
   [-0.14232606  1.1575534  -1.6258671   0.47553247 -0.09795115]]]]


In [41]:
image = torch.randn(1, 1, 6, 6) 
weight = torch.randn(1, 1, 6, 6)

myConvT = torch.from_numpy(myConvT(image.numpy(), weight.numpy()))

torchConvT = conv_transpose2d(image, weight)

myConvT = myConvT.to(torchConvT.dtype)

In [42]:
myConvT,torchConvT

(tensor([[[[-2.1423e-01, -3.8750e-01,  1.0029e+00,  6.9674e-02, -7.5848e-01,
             2.0082e+00,  7.3880e-01,  1.7691e+00,  2.3503e-01,  7.5507e-01,
            -1.3984e-01],
           [ 2.5410e-01,  5.7085e-01,  1.2464e+00, -3.6620e+00,  2.4822e+00,
            -2.9160e+00,  4.9624e+00, -7.8805e-02,  2.8685e+00, -1.5274e+00,
             2.9851e-01],
           [ 2.8582e-01,  1.2329e+00, -6.4030e-01, -1.9927e+00,  3.9974e+00,
            -7.4117e-02,  3.0534e+00,  3.3233e+00, -2.8443e-01,  2.0738e-01,
             1.1614e-01],
           [ 2.6338e+00, -4.6632e+00, -6.2368e-01,  1.3542e+00,  4.1292e+00,
             1.8079e-01,  5.6604e-01, -8.0155e-01,  3.5846e+00,  4.8148e-01,
             4.0726e-01],
           [-6.1977e-01, -2.7843e+00,  2.6734e+00,  5.4689e+00,  6.2441e+00,
            -1.6327e-01,  9.3786e-01,  7.3044e+00, -1.5086e+00,  3.8300e+00,
            -1.1799e+00],
           [-4.7958e+00,  9.9314e-01, -1.5651e+00,  3.4225e+00, -3.1689e+00,
             7.6466e+00